## NLP for Task Classification

**Hypothesis**: Part of Speech (POS) tagging and syntactic dependency parsing provides valuable information for classifying imperative phrases. The thinking is that being able to detect imperative phrases will transfer well to detecting tasks and to-dos.

#### Some Terminology
- [_Imperative mood_](https://en.wikipedia.org/wiki/Imperative_mood) is "used principally for ordering, requesting or advising the listener to do (or not to do) something... also often used for giving instructions as to how to perform a task."
- _Part of speech (POS)_ is a way of categorizing a word based on its syntactic function.
    - The POS tagger from Spacy.io that is used in this notebook differentiates between [*pos_* and *tag_*](https://spacy.io/docs/api/annotation#pos-tagging-english) - *POS (pos_)* refers to "coarse-grained part-of-speech" like `VERB`, `ADJ`, or `PUNCT`; and *POSTAG (tag_)* refers to "fine-grained part-of-speech" like `VB`, `JJ`, or `.`.
- _Syntactic dependency parsing_ is a way of connecting words based on syntactic relationships, [such as](https://spacy.io/docs/api/annotation#dependency-parsing-english) `DOBJ` (direct object), `PREP` (prepositional modifier), or `POBJ` (object of preposition).
    - Check out the dependency parse of the phrase ["Send the report to Kyle by tomorrow"](https://demos.explosion.ai/displacy/?text=Send%20the%20report%20to%20Kyle%20by%20tomorrow&model=en&cpu=1&cph=1) as an example.

### Proposed Features
The imperative mood centers around _actions_, and actions are generally represented in English using verbs. So the features are engineered to also center on the VERB:
1. `FeatureName.VERB`: Does the phrase contain `VERB`(s) of the tag form `VB*`?
2. `FeatureName.FOLLOWING_POS`: Are the words following the `VERB`(s) of certain parts of speech?
3. `FeatureName.FOLLOWING_POSTAG`: Are the words following the `VERB`(s) of certain POS tags?
4. `FeatureName.CHILD_DEP`: Are the `VERB`(s) parents of certain syntactic dependencies?
5. `FeatureName.PARENT_DEP`: Are the `VERB`(s) children of certain syntactic dependencies?
6. `FeatureName.CHILD_POS`: Are the syntactic dependencies that the `VERB`(s) are children of of certain parts of speech?
7. `FeatureName.CHILD_POSTAG`: Are the syntactic dependencies that the `VERB`(s) are children of of certain POS tags?
8. `FeatureName.PARENT_POS`: Are the syntactic dependencies that the `VERB`(s) parent of certain parts of speech?
9. `FeatureName.PARENT_POSTAG`: Are the syntactic dependencies that the `VERB`(s) parent of certain POS tags?

**Notes:**
- Features 2-9 all depend on feature 1 between `True`; if `False`, phrase vectorization will result in all zeroes.
- When features 2-9 are applied to actual phrases, they will append identifying informating about the feature in the form of `_*` (e.g., `FeatureName.FOLLOWING_POSTAG_WRB`).

## Data and Setup

### Building a recipe corpus

I wrote and ran `epicurious_recipes.py`\* to scrape Epicurious.com for recipe instructions and descriptions. I then performed some manual cleanup of the script results. Output is in `epicurious-pos.txt` and `epicurious-neg.txt`.

\* _script (very) loosely based off of https://github.com/benosment/hrecipe-parse_

**Note** that deriving all negative examples in the training set from Epicurious recipe descriptions would result in negative examples that are longer and syntactically more complicated than the positive examples. This is a form of bias.

To (hopefully?) correct for this a bit, I will add the short movie reviews found at https://pythonprogramming.net/static/downloads/short_reviews/ as more negative examples.

This still feels weird because we're selecting negative examples only from specific categories of text (recipe descriptions, short movie reviews) - just because they're readily available. Further, most positive examples are recipe instructions - also a specific (and not necessarily related to the main "task" category) category of text.

Ultimately though, this recipe corpus is a **stopgap/proof of concept** for a corpus more relevant to tasks later on, so I won't worry further about this for now.

In [1]:
import os
from numpy import random

In [2]:
BASE_DIR = os.getcwd()
pos_data_path = BASE_DIR + '/pos.txt'
neg_data_path = BASE_DIR + '/neg.txt'

In [3]:
with open(pos_data_path, 'r', encoding='utf-8') as f:
    pos_data = f.read()
with open(neg_data_path, 'r', encoding='utf-8') as f:
    neg_data = f.read()

In [4]:
pos_data_split = pos_data.split('\n')
neg_data_split = neg_data.split('\n')

num_pos = len(pos_data_split)
num_neg = len(neg_data_split)

# 50/50 split between the number of positive and negative samples
num = num_pos if num_pos < num_neg else num_neg

# shuffle samples
random.shuffle(pos_data_split)
random.shuffle(neg_data_split)

In [5]:
lines = []
for l in pos_data_split[:num]:
    lines.append((l, 'pos'))
for l in neg_data_split[:num]:
    lines.append((l, 'neg'))

In [6]:
# Features as defined in the introduction
from enum import Enum, auto
class FeatureName(Enum):
    VERB = auto()
    FOLLOWING_POS = auto()
    FOLLOWING_POSTAG = auto()
    CHILD_DEP = auto()
    PARENT_DEP = auto()
    CHILD_POS = auto()
    CHILD_POSTAG = auto()
    PARENT_POS = auto()
    PARENT_POSTAG = auto()

## [spaCy.io](https://spacy.io/) for NLP
_Because Stanford CoreNLP is hard to install for Python_

Found Spacy through an article on ["Training a Classifier for Relation Extraction from Medical Literature"](https://www.microsoft.com/developerblog/2016/09/13/training-a-classifier-for-relation-extraction-from-medical-literature/) ([GitHub](https://github.com/CatalystCode/corpus-to-graph-ml))

<img src="nltk_library_comparison.png" alt="NLTK library comparison chart https://spacy.io/docs/api/#comparison" style="width: 400px; margin: 0;"/>

In [7]:
#!conda config --add channels conda-forge
#!conda install spacy
#!python -m spacy download en

### Using the Spacy Data Model for NLP

In [8]:
import spacy
nlp = spacy.load('en')

Spacy's sentence segmentation is lacking... https://github.com/explosion/spaCy/issues/235. So each '\n' will start a new Spacy Doc.

In [9]:
def create_spacy_docs(ll):
    dd = [(nlp(l[0]), l[1]) for l in ll]
    # collapse noun phrases into single compounds
    for d in dd:
        for np in d[0].noun_chunks:
            np.merge(np.root.tag_, np.text, np.root.ent_type_)
    return dd

In [10]:
docs = create_spacy_docs(lines)

### NLP output

Tokenization, POS tagging, and dependency parsing happened automatically with the `nlp(line)` calls above! So let's look at the outputs.

https://spacy.io/docs/usage/data-model and https://spacy.io/docs/api/doc will be useful going forward

In [11]:
for doc in docs[:10]:
    print(list(doc[0].sents))

[Transfer mixture to a medium bowl.]
[Pile the cheese and ham on the unbuttered side of one slice of bread, and put the open-face sandwich in the waffle iron as far away from the hinge as possible.]
[Heat 2 Tbsp. oil in a large nonstick skillet over high.]
[2. Cut the bananas in half and insert a Popsicle stick into each half, as shown.]
[Poach the links in water or lager-style beer until an instant-read thermometer inserted into the center of the sausage registers 145°F/63°C.]
[Pulse chickpeas, pistachios, oil, garlic, lemon zest, salt, and pepper in a food processor until coarsely chopped, about 15 seconds.]
[Put some of the vegetable mixture on each eggplant slice, roll up, and place in the pan.]
[Slice into 1/4" wedges.]
[Scatter crumble, trying not to break it up too much, on a rimmed baking sheet and bake, tossing halfway through, until golden brown, 20–25 minutes.]
[Season steak with 1 tsp. salt and 1 tsp., pepper.]


In [12]:
for doc in docs[:10]:
    print(list(doc[0].noun_chunks))

[Transfer mixture, a medium bowl]
[the cheese, ham, the unbuttered side, one slice, bread, the open-face sandwich, the waffle iron, the hinge]
[Heat 2 Tbsp. oil, a large nonstick skillet]
[the bananas, half, a Popsicle stick, each half]
[the links, water, lager-style beer, an instant-read thermometer, the center, the sausage]
[Pulse chickpeas, a food processor]
[the vegetable mixture, each eggplant slice, place, the pan]
[Slice]
[it, a rimmed baking sheet, bake, golden brown]
[Season steak, 1 tsp, salt, 1 tsp, pepper]


[Spacy's dependency graph visualization](https://demos.explosion.ai/displacy)

In [13]:
for doc in docs[:5]:
    for token in doc[0]:
        print(token.text, token.dep_, token.lemma_, token.pos_, token.tag_, token.head, list(token.children))

Transfer mixture ROOT Transfer mixture NOUN NN Transfer mixture [to, .]
to prep to ADP IN Transfer mixture [a medium bowl]
a medium bowl pobj a medium bowl NOUN NN to []
. punct . PUNCT . Transfer mixture []
Pile ROOT pile VERB VB Pile [the cheese, and, put, .]
the cheese dobj the cheese NOUN NN Pile [and, ham, on, ,]
and cc and CCONJ CC the cheese []
ham conj ham NOUN NN the cheese []
on prep on ADP IN the cheese [the unbuttered side]
the unbuttered side pobj the unbuttered side NOUN NN on [of]
of prep of ADP IN the unbuttered side [one slice]
one slice pobj one slice NOUN NN of [of]
of prep of ADP IN one slice [bread]
bread pobj bread NOUN NN of []
, punct , PUNCT , the cheese []
and cc and CCONJ CC Pile []
put conj put VERB VB Pile [the open-face sandwich, in, away]
the open-face sandwich dobj the open-face sandwich NOUN NN put []
in prep in ADP IN put [the waffle iron]
the waffle iron pobj the waffle iron NOUN NN in []
as advmod as ADV RB away []
far advmod far ADV RB away []
away 

### Featurization

In [14]:
import re
from collections import defaultdict

def featurize(d):
    s_features = defaultdict(int)
    for idx, token in enumerate(d):
        if re.match(r'VB.?', token.tag_) is not None: # note: not using token.pos == VERB because this also includes BES, HVS, MD tags 
            s_features[FeatureName.VERB.name] += 1
            # FOLLOWING_POS
            # FOLLOWING_POSTAG
            next_idx = idx + 1;
            if next_idx < len(d):
                s_features[f'{FeatureName.FOLLOWING_POS.name}_{d[next_idx].pos_}'] += 1
                s_features[f'{FeatureName.FOLLOWING_POSTAG.name}_{d[next_idx].tag_}'] += 1
            # PARENT_DEP
            # PARENT_POS
            # PARENT_POSTAG
            '''
            "Because the syntactic relations form a tree, every word has exactly one head.
            You can therefore iterate over the arcs in the tree by iterating over the words in the sentence."
            https://spacy.io/docs/usage/dependency-parse#navigating
            '''
            if (token.head is not token):
                s_features[f'{FeatureName.PARENT_DEP.name}_{token.head.dep_.upper()}'] += 1
                s_features[f'{FeatureName.PARENT_POS.name}_{token.head.pos_}'] += 1
                s_features[f'{FeatureName.PARENT_POSTAG.name}_{token.head.tag_}'] += 1
            # CHILD_DEP
            # CHILD_POS
            # CHILD_POSTAG
            for child in token.children:
                s_features[f'{FeatureName.CHILD_DEP.name}_{child.dep_.upper()}'] += 1
                s_features[f'{FeatureName.CHILD_POS.name}_{child.pos_}'] += 1
                s_features[f'{FeatureName.CHILD_POSTAG.name}_{child.tag_}'] += 1
    return dict(s_features)

In [15]:
featuresets = [(doc[0], (featurize(doc[0]), doc[1])) for doc in docs]

In [16]:
from statistics import mean, median, mode, stdev
f_lengths = [len(fs[1][0]) for fs in featuresets]

print('Stats on number of features per example:')
print(f'mean: {mean(f_lengths)}')
print(f'stdev: {stdev(f_lengths)}')
print(f'median: {median(f_lengths)}')
print(f'mode: {mode(f_lengths)}')
print(f'max: {max(f_lengths)}')
print(f'min: {min(f_lengths)}')

Stats on number of features per example:
mean: 22.54947689625109
stdev: 14.509311329978114
median: 22.0
mode: 0
max: 75
min: 0


In [17]:
featuresets[:2]

[(Transfer mixture to a medium bowl., ({}, 'pos')),
 (Pile the cheese and ham on the unbuttered side of one slice of bread, and put the open-face sandwich in the waffle iron as far away from the hinge as possible.,
  ({'CHILD_DEP_ADVMOD': 1,
    'CHILD_DEP_CC': 1,
    'CHILD_DEP_CONJ': 1,
    'CHILD_DEP_DOBJ': 2,
    'CHILD_DEP_PREP': 1,
    'CHILD_DEP_PUNCT': 1,
    'CHILD_POSTAG_.': 1,
    'CHILD_POSTAG_CC': 1,
    'CHILD_POSTAG_IN': 1,
    'CHILD_POSTAG_NN': 2,
    'CHILD_POSTAG_RB': 1,
    'CHILD_POSTAG_VB': 1,
    'CHILD_POS_ADP': 1,
    'CHILD_POS_ADV': 1,
    'CHILD_POS_CCONJ': 1,
    'CHILD_POS_NOUN': 2,
    'CHILD_POS_PUNCT': 1,
    'CHILD_POS_VERB': 1,
    'FOLLOWING_POSTAG_NN': 2,
    'FOLLOWING_POS_NOUN': 2,
    'PARENT_DEP_ROOT': 1,
    'PARENT_POSTAG_VB': 1,
    'PARENT_POS_VERB': 1,
    'VERB': 2},
   'pos'))]

On one run, the above line printed the following featureset:
`(Gather foil loosely on top and bake for 1 1/2 hours., ({}, 'pos'))`

This is because the Spacy.io POS tagger provided this:
   `Gather/NNP foil/NN loosely/RB on/IN top/NN and/CC bake/NN for/IN 1 1/2 hours./NNS`

...with no VERBs tagged, which is incorrect.

"Voting - POS taggers and classifiers" in the _Next Steps/Improvements_ section below is meant to improve on this.

---
Compare to [Stanford CoreNLP POS tagger](http://nlp.stanford.edu:8080/corenlp/process):
   `Gather/VB foil/NN loosely/RB on/IN top/JJ and/CC bake/VB for/IN 1 1/2/CD hours/NNS ./.`

And [Stanford Parser](http://nlp.stanford.edu:8080/parser/index.jsp):
   `Gather/NNP foil/VB loosely/RB on/IN top/NN and/CC bake/VB for/IN 1 1/2/CD hours/NNS ./.`

### Classification

In [18]:
random.shuffle(featuresets)

split_num = round(num*2 / 5)

# train and test sets
testing_set = [fs[1] for i, fs in enumerate(featuresets[:split_num])]
training_set =  [fs[1] for i, fs in enumerate(featuresets[split_num:])]

print(f'# training samples: {len(training_set)}')
print(f'# test samples: {len(testing_set)}')

# training samples: 3670
# test samples: 918


In [19]:
# decoupling the functionality of nltk.classify.accuracy
def predict(classifier, gold, prob=True):
    if (prob is True):
        predictions = classifier.prob_classify_many([fs for (fs, ll) in gold])
    else:
        predictions = classifier.classify_many([fs for (fs, ll) in gold])
    return list(zip(predictions, [ll for (fs, ll) in gold]))

def accuracy(predicts, prob=True):
    if (prob is True):
        correct = [label == prediction.max() for (prediction, label) in predicts]
    else:
        correct = [label == prediction for (prediction, label) in predicts]
        
    if correct:
        return sum(correct) / len(correct)
    else:
        return 0

Note below the use of `DummyClassifier` to provide a simple sanity check, a baseline of random predictions. `stratified` means it "generates random predictions by respecting the training set class distribution." (http://scikit-learn.org/stable/modules/model_evaluation.html#dummy-estimators)

> More generally, when the accuracy of a classifier is too close to random, it probably means that something went wrong: features are not helpful, a hyperparameter is not correctly tuned, the classifier is suffering from class imbalance, etc…

If a classifier can beat the `DummyClassifier`, it is at least learning something valuable! How valuable is another question...

In [20]:
from nltk import NaiveBayesClassifier
from nltk.classify.decisiontree import DecisionTreeClassifier
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier

dummy = SklearnClassifier(DummyClassifier(strategy='stratified', random_state=0))
dummy.train(training_set)
dummy_predict = predict(dummy, testing_set)
dummy_accuracy = accuracy(dummy_predict)
print("Dummy classifier accuracy percent:", dummy_accuracy*100)

nb = NaiveBayesClassifier.train(training_set)
nb_predict = predict(nb, testing_set)
nb_accuracy = accuracy(nb_predict)
print("NaiveBayes classifier accuracy percent:", nb_accuracy*100)

multinomial_nb = SklearnClassifier(MultinomialNB())
multinomial_nb.train(training_set)
mnb_predict = predict(multinomial_nb, testing_set)
mnb_accuracy = accuracy(mnb_predict)
print("MultinomialNB classifier accuracy percent:", mnb_accuracy*100)

bernoulli_nb = SklearnClassifier(BernoulliNB())
bernoulli_nb.train(training_set)
bnb_predict = predict(bernoulli_nb, testing_set)
bnb_accuracy = accuracy(bnb_predict)
print("BernoulliNB classifier accuracy percent:", bnb_accuracy*100)

# ??logistic_regression._clf
#   sklearn.svm.LinearSVC : learns SVM models using the same algorithm.
logistic_regression = SklearnClassifier(LogisticRegression())
logistic_regression.train(training_set)
lr_predict = predict(logistic_regression, testing_set)
lr_accuracy = accuracy(lr_predict)
print("LogisticRegression classifier accuracy percent:", lr_accuracy*100)

# ??sgd._clf
#    The 'log' loss gives logistic regression, a probabilistic classifier.
# ??linear_svc._clf
#   can optimize the same cost function as LinearSVC
#   by adjusting the penalty and loss parameters. In addition it requires
#   less memory, allows incremental (online) learning, and implements
#   various loss functions and regularization regimes.
sgd = SklearnClassifier(SGDClassifier(loss='log'))
sgd.train(training_set)
sgd_predict = predict(sgd, testing_set)
sgd_accuracy = accuracy(sgd_predict)
print("SGD classifier accuracy percent:", sgd_accuracy*100)

# slow
# using libsvm with kernel 'rbf' (radial basis function)
svc = SklearnClassifier(SVC(probability=True))
svc.train(training_set)
svc_predict = predict(svc, testing_set)
svc_accuracy = accuracy(svc_predict)
print("SVC classifier accuracy percent:", svc_accuracy*100)

# ??linear_svc._clf
#    Similar to SVC with parameter kernel='linear', but implemented in terms of
#    liblinear rather than libsvm, so it has more flexibility in the choice of
#    penalties and loss functions and should scale better to large numbers of
#    samples.
#    Prefer dual=False when n_samples > n_features.
linear_svc = SklearnClassifier(LinearSVC(dual=False))
linear_svc.train(training_set)
linear_svc_predict = predict(linear_svc, testing_set, False)
linear_svc_accuracy = accuracy(linear_svc_predict, False)
print("LinearSVC classifier accuracy percent:", linear_svc_accuracy*100)

# slower
dt = DecisionTreeClassifier.train(training_set)
dt_predict = predict(dt, testing_set, False)
dt_accuracy = accuracy(dt_predict, False)
print("DecisionTree classifier accuracy percent:", dt_accuracy*100)

random_forest = SklearnClassifier(RandomForestClassifier(n_estimators = 100))
random_forest.train(training_set)
rf_predict = predict(random_forest, testing_set)
rf_accuracy = accuracy(rf_predict)
print("RandomForest classifier accuracy percent:", rf_accuracy*100)

Dummy classifier accuracy percent: 50.98039215686274
NaiveBayes classifier accuracy percent: 67.75599128540306
MultinomialNB classifier accuracy percent: 79.30283224400871
BernoulliNB classifier accuracy percent: 72.87581699346404
LogisticRegression classifier accuracy percent: 81.15468409586056
SGD classifier accuracy percent: 80.93681917211329
SVC classifier accuracy percent: 79.30283224400871
LinearSVC classifier accuracy percent: 81.15468409586056
DecisionTree classifier accuracy percent: 75.81699346405229
RandomForest classifier accuracy percent: 80.71895424836602


### SGD: Multiple Epochs

`sgd` classifiers improves with epochs. `??sgd._clf` tells us that the default number of epochs `n_iter` is 5. So let's run more epochs. Also not that the training_set shuffle is `True` by default.

In [21]:
num_epochs = 1000
sgd = SklearnClassifier(SGDClassifier(loss='log', n_iter=num_epochs))
sgd.train(training_set)
sgd_predict = predict(sgd, testing_set)
sgd_accuracy = accuracy(sgd_predict)
print(f"SGDClassifier classifier accuracy percent (epochs: {num_epochs}):", sgd_accuracy*100)

SGDClassifier classifier accuracy percent (epochs: 1000): 81.15468409586056


Fortunately, 1000 epochs run very quickly! And `SGDClassifier` performance has improved with more iterations.

*Also note that we can set `warm_start` to `True` if we want to take advantage of online learning and reuse the solution of the previous call.*

### Analysis

We're going to scope analysis down to our top-performing classifiers, which consistently perform with >80% accuracy: `LogisticRegression`, `SVC`, `LinearSVC`, `SGD`, and `RandomForest`.

We'll also include `Dummy` as a baseline.

**Note**: I considered omitting `LinearSVC` since we have `SVC` also performing with high accuracy, and since `LinearSVC` does not provide probability estimates I rely on a lot during this analysis. However, `LinearSVC` is much faster than `SVC` and is meant to "scale better to large numbers of samples."

#### Most Informative Features

In [22]:
# https://stackoverflow.com/a/11140887
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:round(n/2)], coefs_with_fns[:-(round(n/2) + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [23]:
print('SGD')
show_most_informative_features(sgd._vectorizer, sgd._clf, 15)
print()
print('Logistic Regression')
show_most_informative_features(logistic_regression._vectorizer, logistic_regression._clf, 15)
print()
print('LinearSVC')
show_most_informative_features(linear_svc._vectorizer, linear_svc._clf, 15)

SGD
	-2.6326	CHILD_DEP_AGENT		1.9577	CHILD_DEP_ADVMOD||CONJ
	-2.1426	CHILD_POSTAG_HYPH		1.8642	FOLLOWING_POSTAG_JJS
	-2.0694	CHILD_POSTAG_``		1.7767	CHILD_DEP_NPADVMOD
	-1.9954	CHILD_DEP_INTJ 		1.7219	PARENT_POS_PROPN
	-1.8781	PARENT_DEP_MARK		1.7219	PARENT_POSTAG_NNP
	-1.6606	CHILD_POS_ADJ  		1.6002	CHILD_POSTAG_-LRB-
	-1.6397	CHILD_DEP_POBJ||PREP		1.5037	PARENT_DEP_OPRD
	-1.5879	CHILD_POSTAG_JJS		1.4823	PARENT_DEP_DEP 

Logistic Regression
	-2.1170	CHILD_DEP_AGENT		1.5115	CHILD_DEP_NPADVMOD
	-1.8526	CHILD_POSTAG_HYPH		1.3864	PARENT_POS_PROPN
	-1.4606	CHILD_DEP_NSUBJ		1.3864	PARENT_POSTAG_NNP
	-1.3723	CHILD_POS_ADJ  		1.3238	CHILD_POSTAG_-LRB-
	-1.3251	PARENT_POSTAG_VBZ		1.3058	CHILD_DEP_ADVMOD||CONJ
	-1.3070	CHILD_DEP_ATTR 		1.1697	CHILD_POSTAG_-RRB-
	-1.2565	CHILD_DEP_INTJ 		1.1154	VERB           
	-1.2104	CHILD_POSTAG_``		0.9758	PARENT_DEP_DEP 

LinearSVC
	-1.3895	PARENT_DEP_MARK		1.7336	FOLLOWING_POSTAG_JJS
	-1.3817	CHILD_DEP_POBJ||PREP		1.0026	CHILD_DEP_META 
	-1.2403	CHILD_POSTA

*Note: Because `SVC` is using the nonlinear RBF kernel, we cannot show the most informative features (`coef_ is only available when using a linear kernel`). The same applies for `Random Forest`.*

In [24]:
spacy.explain("JJS")

'adjective, superlative'

**Negative coefficients**:
- VERB parents [`AGENT`](http://universaldependencies.org/docs/sv/dep/nmod-agent.html): "used for agents of passive verbs" - interpreting this to mean that _existence of passive verbs (i.e., the opposite of active verbs) means negative correlation with it being imperative_
- VERB followed by a `WRB`: "wh-adverb" (where, when)
- VERB is a child of [`AMOD`](http://universaldependencies.org/en/dep/amod.html): "any adjective or adjectival phrase that serves to modify the meaning" of the verb


**Positive coefficients**:
- VERB parents a `-RRB-`: "right round bracket"
- VERB is a child of `PROPN`: "proper noun"
- VERB is a child of `NNP`: "noun, proper singular"

#### Scikit Learn metrics: Confusion matrix, Classification report, F1 score, Log loss

http://scikit-learn.org/stable/modules/model_evaluation.html

In [25]:
from sklearn import metrics

def classification_report(predict, prob=True):
    predictions, labels = zip(*predict)
    if prob is True:
        return metrics.classification_report(labels, [p.max() for p in predictions])
    else:
        return metrics.classification_report(labels, predictions)

def confusion_matrix(predict, prob=True, print_layout=False):
    predictions, labels = zip(*predict)
    if print_layout is True:
        print('Layout\n[[tn   fp]\n [fn   tp]]\n')
    if prob is True:
        return metrics.confusion_matrix(labels, [p.max() for p in predictions])
    else:
        return metrics.confusion_matrix(labels, predictions)

def log_loss(predict):
    predictions, labels = zip(*predict)
    return metrics.log_loss(labels, [p.prob('pos') for p in predictions])

def roc_auc_score(predict):
    predictions, labels = zip(*predict)
    # need to convert labels to binary classification of 0 or 1
    return metrics.roc_auc_score([1 if l == 'pos' else 0 for l in labels], [p.prob('pos') for p in predictions], average='weighted')

In [26]:
print('SGD')
print(classification_report(sgd_predict))
print()
print('Logistic Regression')
print(classification_report(lr_predict))
print()
print('SVC')
print(classification_report(svc_predict))
print()
print('LinearSVC')
print(classification_report(linear_svc_predict, False))
print()
print('Random Forest')
print(classification_report(rf_predict))

SGD
             precision    recall  f1-score   support

        neg       0.87      0.73      0.79       455
        pos       0.77      0.89      0.83       463

avg / total       0.82      0.81      0.81       918


Logistic Regression
             precision    recall  f1-score   support

        neg       0.87      0.73      0.79       455
        pos       0.77      0.89      0.83       463

avg / total       0.82      0.81      0.81       918


SVC
             precision    recall  f1-score   support

        neg       0.83      0.74      0.78       455
        pos       0.77      0.85      0.80       463

avg / total       0.80      0.79      0.79       918


LinearSVC
             precision    recall  f1-score   support

        neg       0.87      0.73      0.79       455
        pos       0.77      0.89      0.83       463

avg / total       0.82      0.81      0.81       918


Random Forest
             precision    recall  f1-score   support

        neg       0.84      0.

In [27]:
print('Layout\n[[tn   fp]\n [fn   tp]]\n')

print('SGD')
print(confusion_matrix(sgd_predict))
print()
print('Logistic Regression')
print(confusion_matrix(lr_predict))
print()
print('SVC')
print(confusion_matrix(svc_predict))
print()
print('LinearSVC')
print(confusion_matrix(linear_svc_predict, False))
print()
print('Random Forest')
print(confusion_matrix(rf_predict))

Layout
[[tn   fp]
 [fn   tp]]

SGD
[[334 121]
 [ 52 411]]

Logistic Regression
[[333 122]
 [ 51 412]]

SVC
[[336 119]
 [ 71 392]]

LinearSVC
[[332 123]
 [ 50 413]]

Random Forest
[[343 112]
 [ 65 398]]


The lower the better for `log_loss`...

In [28]:
print(f'SGD: {log_loss(sgd_predict)}')
print(f'Logistic Regression: {log_loss(lr_predict)}')
print(f'SVC: {log_loss(svc_predict)}')
print(f'Random Forest: {log_loss(rf_predict)}')

SGD: 0.3935142288286698
Logistic Regression: 0.39304204803084514
SVC: 0.4554477344960195
Random Forest: 0.4778529494158974


The higher the better for `roc_auc_score`...

In [29]:
print(f'SGD: {roc_auc_score(sgd_predict)}')
print(f'Logistic Regression: {roc_auc_score(lr_predict)}')
print(f'SVC: {roc_auc_score(svc_predict)}')
print(f'Random Forest: {roc_auc_score(rf_predict)}')

SGD: 0.9099494458025774
Logistic Regression: 0.9093750741698905
SVC: 0.8596610732679846
Random Forest: 0.8954358816129875


*Note: We cannot compute `log_loss` or `roc_auc_score` for `LinearSVC` because it does not provide probability estimates.*

#### Performance on sample tasks

In [30]:
sample_tasks = ["Mow lawn", "Mow the lawn", "Buy new shoes", "Feed the dog", "Send report to Kyle", "Send the report to Kyle", "Peel the potatoes"]
features = [featurize(nlp(task)) for task in sample_tasks]

tasks_dummy = [(l, p.prob('pos')*1.0) for l, p in zip(dummy.classify_many(features), dummy.prob_classify_many(features))]
tasks_logistic = [(l, p.prob('pos')) for l,p in zip(logistic_regression.classify_many(features), logistic_regression.prob_classify_many(features))]
tasks_svc = [(l, p.prob('pos')) for l,p in zip(svc.classify_many(features), svc.prob_classify_many(features))]
tasks_linear_svc = linear_svc.classify_many(features)
tasks_sgd = [(l, p.prob('pos')) for l,p in zip(sgd.classify_many(features), sgd.prob_classify_many(features))]
tasks_rf = [(l, p.prob('pos')) for l,p in zip(random_forest.classify_many(features), random_forest.prob_classify_many(features))]

print(f'Dummy: {tasks_dummy}')
print(f'LogisticRegression: {tasks_logistic}')
print(f'SVC: {tasks_svc}')
print(f'LinearSVC: {tasks_linear_svc}')
print(f'SGD: {tasks_sgd}')
print(f'Random Forest: {tasks_rf}')

Dummy: [('pos', 1.0), ('pos', 1.0), ('pos', 1.0), ('pos', 1.0), ('neg', 0.0), ('pos', 1.0), ('neg', 0.0)]
LogisticRegression: [('pos', 0.60536063873754009), ('pos', 0.78429219579976917), ('pos', 0.92523718226466312), ('pos', 0.78429219579976917), ('pos', 0.83207193305531935), ('pos', 0.78429219579976917), ('pos', 0.80650010865428312)]
SVC: [('pos', 0.79159760798143863), ('pos', 0.7681397213228357), ('pos', 0.83605640813131132), ('pos', 0.7681397213228357), ('pos', 0.83634212180267864), ('pos', 0.7681397213228357), ('pos', 0.80603110717753113)]
LinearSVC: ['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
SGD: [('pos', 0.60284861642010268), ('pos', 0.80083526334838107), ('pos', 0.93808575735324451), ('pos', 0.80083526334838107), ('pos', 0.85845318013153771), ('pos', 0.80083526334838107), ('pos', 0.81958067142169999)]
Random Forest: [('pos', 0.54456501433002391), ('pos', 0.81323406399321774), ('pos', 0.79966535735210864), ('pos', 0.81323406399321774), ('pos', 0.73999999999999999), ('pos',

_Note: `LinearSVC` is not implemented to provide probability estimates (https://github.com/scikit-learn/scikit-learn/issues/4820)_

## Next Steps and Improvements

1. Training set may be too specific/not relevant enough (recipe instructions for positive dataset, recipe descriptions+short movie reviews for negative dataset)
2. Throwing features into a blender - need to understand value of each
    - What feature "classes" tend to perform the best/worst?
    - [PCA](http://jotterbach.github.io/2016/03/24/Principal_Component_Analysis/): Reducing dimensionality using most informative feature information
3. Phrase vectorizations of all 0s - is this problematic?
4. Varying feature vector lengths - does this matter?
5. Voting - POS taggers and classifiers
    - Ensembles (http://scikit-learn.org/stable/modules/ensemble.html)
6. Combining verb phrases
7. Cross validation, grid search
8. Look at examples from different quadrants of the confusion matrix - is there something we can learn?
    - Same idea with the classification report

---
# Things abandoned

## NLTK

I needed a library that supports dependency parsing, which NLTK does not... so I thought I'd add the [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/) toolkit and [its associated software](https://nlp.stanford.edu/software/) to NLTK. However, there are many conflicting instructions for installing the Java-based project, depending on NLTK version used. By the time I figured this out, the installation had become a time sink. So I abandoned this effort in favor of Spacy.io.

I might return this way if I want to improve results/implement a voter system between the various linguistic and classification methods later.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

### Tokenization

In [ ]:
sentences = [s for l in lines for s in sent_tokenize(l)] # punkt
sentences

In [ ]:
tagged_sentences = []
for s in sentences:
    words = word_tokenize(s)
    tagged = nltk.pos_tag(words) # averaged_perceptron_tagger
    tagged_sentences.append(tagged)
print(tagged_sentences)

#### Note: POS accuracy

`Run down to the shop, will you, Peter` is parsed unexpectedly by `nltk.pos_tag`:
> `[('Run', 'NNP'), ('down', 'RB'), ('to', 'TO'), ('the', 'DT'), ('shop', 'NN'), (',', ','), ('will', 'MD'), ('you', 'PRP'), (',', ','), ('Peter', 'NNP')]`

`Run` is tagged as a `NNP (proper noun, singular)`

I expected an output more like what the [Stanford Parser](http://nlp.stanford.edu:8080/parser/) provides:
> `Run/VBG down/RP to/TO the/DT shop/NN ,/, will/MD you/PRP ,/, Peter/NNP`

`Run` is tagged as a `VGB (verb, gerund/present participle)` - still not quite the `VB` I want, but at least it's a `V*`

_MEANWHILE..._

`nltk.pos_tag` did better with:
> `[('Do', 'VB'), ('not', 'RB'), ('clean', 'VB'), ('soot', 'NN'), ('off', 'IN'), ('the', 'DT'), ('window', 'NN')]`

Compared to [Stanford CoreNLP](http://nlp.stanford.edu:8080/corenlp/process) (note that this is different than what [Stanford Parser](http://nlp.stanford.edu:8080/parser/) outputs):
> `(ROOT (S (VP (VB Do) (NP (RB not) (JJ clean) (NN soot)) (PP (IN off) (NP (DT the) (NN window))))))`

Concern: _clean_ as `VB (verb, base form)` vs `JJ (adjective)` 

**IMPROVE** POS taggers should vote: nltk.pos_tag (averaged_perceptron_tagger), Stanford Parser, CoreNLP, etc.

Note what Spacy POS tagger did with `Run down to the shop, will you Peter`:

`Run/VB down/RP to/IN the shop/NN ,/, will/MD you/PRP ,/, Peter/NNP`

    where `Run` is the `VB` I expected from POS tagging (compared to `nltk.pos_tag` result of `NNP`). Also note that Spacy collapses `the shop` into a single unit, which should be helpful during featurization.

### Featurization

In [ ]:
import re
from collections import defaultdict

featuresets = []
for ts in tagged_sentences:
    s_features = defaultdict(int)
    for idx, tup in enumerate(ts):
        #print(tup)
        pos = tup[1]
        # FeatureName.VERB
        is_verb = re.match(r'VB.?', pos) is not None
        print(tup, is_verb)
        if is_verb:
            s_features[FeatureName.VERB] += 1
            # FOLLOWING_POS
            next_idx = idx + 1;
            if next_idx < len(ts):
                s_features[f'{FeatureName.FOLLOWING}_{ts[next_idx][1]}'] += 1
            # VERB_MODIFIER
            # VERB_MODIFYING
        else:
            s_features[FeatureName.VERB] = 0
    featuresets.append(dict(s_features))

print()
print(featuresets)

### [Stanford NLP](https://nlp.stanford.edu/software/)
Setup guide used: https://stackoverflow.com/a/34112695

In [ ]:
# Get dependency parser, NER, POS tagger
!wget https://nlp.stanford.edu/software/stanford-parser-full-2017-06-09.zip
!wget https://nlp.stanford.edu/software/stanford-ner-2017-06-09.zip
!wget https://nlp.stanford.edu/software/stanford-postagger-full-2017-06-09.zip
!unzip stanford-parser-full-2017-06-09.zip
!unzip stanford-ner-2017-06-09.zip
!unzip stanford-postagger-full-2017-06-09.zip

In [ ]:
from nltk.parse.stanford import StanfordParser
from nltk.parse.stanford import StanfordDependencyParser
from nltk.parse.stanford import StanfordNeuralDependencyParser
from nltk.tag.stanford import StanfordPOSTagger, StanfordNERTagger
from nltk.tokenize.stanford import StanfordTokenizer